## this is graph test 3
## test on K-nn graph but exist all vertexs, means single vertex exist

In [1]:
import collections
import os.path as osp
import torch
from torch_geometric.data import Data
from torch_geometric.data import InMemoryDataset
import numpy as np
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from itertools import repeat, product
import time
from get_dataset_whole import *
from get_dataset3 import *
from torch_geometric.nn import ChebConv
from torch_geometric.nn import GCNConv

In [2]:
torch.cuda.is_available()

True

In [11]:
# def read_ima(filepath):
#     ima_list = []
#     father_path = os.listdir(filepath)
#     for allDir in father_path:
#         child_path = os.path.join(filepath, allDir)
#         ima_list.append(child_path)
#     return ima_list

In [13]:
# '''build single graph each image, store the matrix'''
# filepath = '/home/dan/Aiki_data/data/block_matrix/AftMid3r_test/'
# aftPath = '/home/dan/Aiki_data/data/block_matrix/single_graph3r_test/'
# files = read_ima(filepath)
# for file in files:
#     big_matrix = np.load(file)
# #     print(big_matrix.shape)
#     for num_matrix in range(int(big_matrix.shape[0] / 225)):
#         each_matrix = big_matrix[num_matrix*225:(num_matrix+1)*225, :]
#         file_name = osp.split(file)[1]
        
# #         print(file_name.split('matrix',1)[0])
#         np.save(aftPath+file_name.split('matrix',1)[0] + str(num_matrix)+'.npy', each_matrix)
        
# print('finish')

finish


In [4]:
'''train_dataset'''
matrix_filepath = '/media/kamatalab/78cde73a-a99c-4bcc-b0af-7ba8c7da32f3/dan/Aiki_data/data/block_matrix/resize_120/singal_graph3r'
connect_filepath = '/media/kamatalab/78cde73a-a99c-4bcc-b0af-7ba8c7da32f3/dan/Aiki_data/data/connection_np/singal_graph3r'

Datalist = Get_DataList(matrix_filepath, connect_filepath)
datalist = Datalist.Data_build()

ValueError: invalid literal for int() with base 10: '_'

In [7]:
'''test dataset'''
matrix_filepath_test = '/home/dan/Aiki_data/data/block_matrix/single_graph3r_test'
connect_filepath_test = '/home/dan/Aiki_data/data/connection_np/singal_graph3r_test'

Datalist_test = Get_DataList(matrix_filepath_test, connect_filepath_test)
datalist_test = Datalist_test.Data_build()

2478

In [8]:
dataset_train = MyOwnDataset(root='/home/dan/Aiki_data/data/Dataset/single_graph_3rwhole', Datalist = datalist)

In [9]:
dataset_test = MyOwnDataset(root='/home/dan/Aiki_data/data/Dataset/single_graph3r_test', Datalist = datalist_test)

In [10]:
'''since the dataset is orderly formed, it should be shuffled'''
dataset_train.shuffle()
dataset_test.shuffle()

MyOwnDataset(618)

In [11]:
'''try batch'''
loader_train = DataLoader(dataset_train, batch_size = 32, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size = 32, shuffle=True)

In [12]:
for loader in loader_train:
    print(loader)
    print(loader.batch.shape)
    break

Batch(batch=[7200], edge_index=[2, 19290], x=[7200, 64], y=[32])
torch.Size([7200])


In [19]:
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import graclus

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = ChebConv(dataset_train.num_node_features, 16, K=4)
        self.conv2 = ChebConv(16, int(dataset_train.num_classes),K=4)
        
#         self.global_pool = global_mean_pool()

#         self.fc1 = nn.Linear(280, 512)
#         self.fc2 = nn.Linear(512, 5)
#         self.conv1 = GCNConv(dataset_train.num_node_features, 16)
#         self.conv2 = GCNConv(16, int(dataset_train.num_classes))


    def graph_max_pool(self, x, p):
        x = torch.unsqueeze(x, 0)  # 增加一个维度, batch_size
        if p > 1: 
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p          
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x.squeeze(0) 
        else:
            return x.squeeze(0)
    
    def graph_fc(self, x, fcout):
        fcin = x.size(0)
        fc_oper = nn.Linear(fcin, fcout)
        x = fc_oper(x)
        return x
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # CL1 like-wise
        x = self.conv1(x, edge_index)
#         print('CL1 x.shape',x.shape)
        x = F.relu(x)
        
        # CL2 like-wise
#         print(edge_index.shape)
        x = self.conv2(x, edge_index)
#         print(x.shape)
        x = F.relu(x)
#         x = self.graph_max_pool(x, 2)
#         # FC1 like-wise
#         x = self.graph_max_pool(x, 2)
        print('before the pool ', x.shape)

#         x = global_mean_pool(x, data.batch)
        x = graclus(edge_index)

        
        print('after global_mean_pool ', x.shape)
#         x = x.view(-1)
        
#         lin = nn.Linear(x.size(0),512).to(device)
#         x = lin(x)
#         x = self.graph_fc(x, 512).cuda()
#         x = self.fc1(x)

        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # FC2-wise
#         lin = nn.Linear(x.size(0),5).to(device)
#         x = lin(x)
#         x = self.graph_fc(x, 5).cuda()
#         x = self.fc2(x)
#         print('final output shape', x.shape)
#         return F.log_softmax(x, dim=1)
        return x

    def loss(self, y, y_target):
#         print('y shape', y.shape)
#         print('y_target shape',y_target.shape)
        '''extend the dimsion of y, adding minibatch dim'''
#         y = torch.unsqueeze(y, 0)
        loss = nn.CrossEntropyLoss()(y,y_target)

        l2_loss = 0.0
        for param in self.parameters():
            data = param* param
            l2_loss += data.sum()
           
        loss += 0.5* 5e-4 * l2_loss
            
        return loss
    
    def evaluation(self, y_predicted, y_target):
#         print('y_predicted is',y_predicted)
#         print('y_target is ',y_target)
        return_val = 0
        _, class_predicted = torch.max(y_predicted,1)
        
#         print(class_predicted)
#         print(y_target)
        

        correc_num = (class_predicted == (y_target.data)).sum()
#         print(correc_num.data)
#         print('class_predicted is ', class_predicted)
#         return 100.0* (class_predicted == float(y_target.data)).sum()/ float(y_predicted.size(0))
#         if class_predicted == y_target:
#             return_val = 1
#             return return_val
            
#         else:
#             return return_val
        return int(correc_num.data)

In [20]:
'''test on batch'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
batch_size = 1
model.train()
index_num = 0
for epoch in range(200):
    
    running_loss = 0.0
    
    running_accuracy = 0
#     inner_accuracy = 0
    
    running_total = 0
#     inner_total = 0
    test_loss = 0
    test_acc = 0
    test_total = 0
    
    t_start = time.time()
    model.train()
    for loader in loader_train:
        optimizer.zero_grad()
        
        data = loader.to(device)
        out = model(data)
#         print('out is ',out)
#         print('data.y is', data.y)

        loss = model.loss(out,data.y)

        acc_num = model.evaluation(out,data.y)
#         print('num is shah ',acc_num)
        
        break
        running_loss += loss
        running_accuracy += acc_num
        running_total += 32

        
#         if running_total % 200 == 0:
#             print('epoch= %d, i= %4d, loss(batch)= %.4f, accuray(batch)= %.2f' 
#                   % (epoch+1, running_total, loss, running_accuracy/running_total))

        loss.backward()
        optimizer.step()
    t_stop = time.time() - t_start
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f' % 
          (epoch+1, running_loss/running_total, running_accuracy/running_total, t_stop))
    break
    model.eval()
    for loader in loader_test:
        data = loader.to(device)
        out = model(data)
        loss = model.loss(out,data.y)
        acc_num = model.evaluation(out,data.y)
        test_loss += loss
        test_acc += acc_num
        test_total += 32 
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f' % 
          (epoch+1, test_loss/test_total, test_acc/test_total))

RuntimeError: CUDA error: device-side assert triggered

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
batch_size = 1
model.train()
index_num = 0
for epoch in range(200):
    
    running_loss = 0.0
    
    running_accuracy = 0
#     inner_accuracy = 0
    
    running_total = 0
#     inner_total = 0
    test_loss = 0
    test_acc = 0
    test_total = 0
    
    t_start = time.time()
    model.train()
    for index_num in range(len(dataset_train)):
        optimizer.zero_grad()
        
        data = dataset_train[index_num].to(device)
        out = model(data)
#         print('out is ',out)
#         print('data.y is', data.y)
        
        loss = model.loss(out,data.y)

        acc_num = model.evaluation(out,data.y)
        
        running_loss += loss
        running_accuracy += acc_num
#         inner_accuracy += acc_num
        running_total += 1
#         inner_total += 1
        
#         if running_total % 200 == 0:
#             print('epoch= %d, i= %4d, loss(batch)= %.4f, accuray(batch)= %.2f' 
#                   % (epoch+1, running_total, loss, running_accuracy/running_total))

#         break
        loss.backward()
        optimizer.step()
    t_stop = time.time() - t_start
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f, time= %.3f' % 
          (epoch+1, running_loss/running_total, running_accuracy/running_total, t_stop))
#     break
    model.eval()
    for index_num_test in range(len(dataset_test)):
        data = dataset_test[index_num_test].to(device)
        out = model(data)
        loss = model.loss(out,data.y)
        acc_num = model.evaluation(out,data.y)
        test_loss += loss
        test_acc += acc_num
        test_total += 1 
    print('epoch= %d, loss(train)= %.3f, accuracy(train)= %.3f' % 
          (epoch+1, test_loss/test_total, test_acc/test_total))

epoch= 1, loss(train)= 4.484, accuracy(train)= 0.988, time= 22.272
epoch= 1, loss(train)= 7.432, accuracy(train)= 0.126
epoch= 2, loss(train)= 0.512, accuracy(train)= 0.895, time= 21.977
epoch= 2, loss(train)= 5.813, accuracy(train)= 0.126
epoch= 3, loss(train)= 0.600, accuracy(train)= 0.870, time= 22.117
epoch= 3, loss(train)= 527.480, accuracy(train)= 0.126
epoch= 4, loss(train)= 0.957, accuracy(train)= 0.914, time= 22.033
epoch= 4, loss(train)= 5.472, accuracy(train)= 0.126
epoch= 5, loss(train)= 0.431, accuracy(train)= 0.916, time= 22.280
epoch= 5, loss(train)= 5.415, accuracy(train)= 0.126
epoch= 6, loss(train)= 0.394, accuracy(train)= 0.918, time= 22.322
epoch= 6, loss(train)= 6.445, accuracy(train)= 0.126
epoch= 7, loss(train)= 0.376, accuracy(train)= 0.926, time= 22.177
epoch= 7, loss(train)= 6.323, accuracy(train)= 0.126
epoch= 8, loss(train)= 0.309, accuracy(train)= 0.941, time= 22.032
epoch= 8, loss(train)= 6.557, accuracy(train)= 0.126
epoch= 9, loss(train)= 0.398, accuracy

epoch= 68, loss(train)= 1785.582, accuracy(train)= 0.126
epoch= 69, loss(train)= 1.725, accuracy(train)= 0.944, time= 21.707
epoch= 69, loss(train)= 5.346, accuracy(train)= 0.126
epoch= 70, loss(train)= 0.541, accuracy(train)= 0.911, time= 22.202
epoch= 70, loss(train)= 4.859, accuracy(train)= 0.126
epoch= 71, loss(train)= 0.618, accuracy(train)= 0.855, time= 21.869
epoch= 71, loss(train)= 5.300, accuracy(train)= 0.126
epoch= 72, loss(train)= 0.606, accuracy(train)= 0.852, time= 21.287
epoch= 72, loss(train)= 6.081, accuracy(train)= 0.126
epoch= 73, loss(train)= 0.606, accuracy(train)= 0.860, time= 22.092
epoch= 73, loss(train)= 5.506, accuracy(train)= 0.126
epoch= 74, loss(train)= 0.557, accuracy(train)= 0.868, time= 21.984
epoch= 74, loss(train)= 6.195, accuracy(train)= 0.126
epoch= 75, loss(train)= 0.511, accuracy(train)= 0.882, time= 21.896
epoch= 75, loss(train)= 339.072, accuracy(train)= 0.126
epoch= 76, loss(train)= 17.220, accuracy(train)= 0.954, time= 21.067
epoch= 76, loss(tr

epoch= 135, loss(train)= 0.467, accuracy(train)= 0.894, time= 21.029
epoch= 135, loss(train)= 6.807, accuracy(train)= 0.126
epoch= 136, loss(train)= 1.408, accuracy(train)= 0.941, time= 21.211
epoch= 136, loss(train)= 5702.866, accuracy(train)= 0.126
epoch= 137, loss(train)= 5.370, accuracy(train)= 0.930, time= 21.050
epoch= 137, loss(train)= 5.812, accuracy(train)= 0.126
epoch= 138, loss(train)= 0.705, accuracy(train)= 0.820, time= 21.101
epoch= 138, loss(train)= 5.204, accuracy(train)= 0.126
epoch= 139, loss(train)= 0.867, accuracy(train)= 0.767, time= 21.125
epoch= 139, loss(train)= 4.566, accuracy(train)= 0.126
epoch= 140, loss(train)= 0.743, accuracy(train)= 0.800, time= 21.053
epoch= 140, loss(train)= 5.931, accuracy(train)= 0.126
epoch= 141, loss(train)= 0.585, accuracy(train)= 0.863, time= 21.029
epoch= 141, loss(train)= 5.969, accuracy(train)= 0.126
epoch= 142, loss(train)= 0.520, accuracy(train)= 0.893, time= 20.989
epoch= 142, loss(train)= 7.981, accuracy(train)= 0.126
epoch

In [37]:
print(model)

Net(
  (conv1): ChebConv(64, 16, K=4, normalization=sym)
  (conv2): ChebConv(16, 5, K=4, normalization=sym)
  (fc1): Linear(in_features=280, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=5, bias=True)
)


In [36]:
'''make the test'''
model.eval()
test_total = 0
test_acc = 0
for index_num in range(len(dataset_test)):
    data = dataset_test[index_num].to(device)
    out = model(data)
    
    acc_num = model.evaluation(out,data.y)
    test_acc += acc_num
    test_total += 1

print('accuracy is %d, out of %d'%(test_acc/test_total, test_total))
print(test_acc)

accuracy is 0, out of 618
78
